In [ ]:
import pandas as pd

data = pd.read_csv("../data/posts_hashtags_selected_columns.csv")

In [9]:
data

,id,bucketUrl,text,textLanguage,createTimeISO,isAd,author_nickName,author_signature,author_fans,video_duration,...,shareCount,playCount,collectCount,commentCount,isSponsored,hashtag_name,hashtag_title,searchHashtag_name,searchHashtag_views,location_address
0,7472461334404238614,https://storage.cloud.google.com/capstone-soci...,Straffer Hals mit diesen Schritten 👍 #halsfalt...,de,2025-02-17T18:46:26.000Z,False,Face Fitness Anti Aging,Falten Frei ohne Botox\nAnti Aging mit eigenen...,233400,112,...,15,6113,121,1,False,halsfalten,NaN,hautpflege,2700000000,"Marienplatz, 80331 München, Germany"
1,7472461334404238614,https://storage.cloud.google.com/capstone-soci...,Straffer Hals mit diesen Schritten 👍 #halsfalt...,de,2025-02-17T18:46:26.000Z,False,Face Fitness Anti Aging,Falten Frei ohne Botox\nAnti Aging mit eigenen...,233400,112,...,15,6113,121,1,False,hautpflege,NaN,hautpflege,2700000000,"Marienplatz, 80331 München, Germany"
2,7472461334404238614,https://storage.cloud.google.com/capstone-soci...,Straffer Hals mit diesen Schritten 👍 #halsfalt...,de,2025-02-17T18:46:26.000Z,False,Face Fitness Anti Aging,Falten Frei ohne Botox\nAnti Aging mit eigenen...,233400,112,...,15,6113,121,1,False,facefitness,NaN,hautpflege,2700000000,"Marienplatz, 80331 München, Germany"
3,7472533937667968278,https://storage.cloud.google.com/capstone-soci...,Aquafacial🫧 Boost für deine Haut💆🏻‍♀️ Tief ger...,de,2025-02-17T23:28:17.000Z,False,SKINID_STUDIO,all about ✨Aquafacial & Microneedling✨ \nat @k...,32,10,...,1,940,2,2,False,appointment,NaN,hautpflege,2700000000,"Nurnberg, Bavaria, Germany"
4,7472533937667968278,https://storage.cloud.google.com/capstone-soci...,Aquafacial🫧 Boost für deine Haut💆🏻‍♀️ Tief ger...,de,2025-02-17T23:28:17.000Z,False,SKINID_STUDIO,all about ✨Aquafacial & Microneedling✨ \nat @k...,32,10,...,1,940,2,2,False,treatment,NaN,hautpflege,2700000000,"Nurnberg, Bavaria, Germany"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12266,7459833919379131678,NaN,#fypシ #skincare #skincareroutine #storytimes #...,un,2025-01-14T18:05:53.000Z,False,Alchemy,NaN,35900,60,...,94,277200,594,75,False,tiktok,NaN,skincareroutine,155600000000,NaN
12267,7335544341332643118,NaN,My mom rates my skincare routine👀🥴,en,2024-02-14T19:39:09.000Z,False,Theresa Krug,✨Lifestyle • DIY • Skincare/Makeup✨\nFlorida📍🌴...,2900000,60,...,18,162100,738,58,False,NaN,NaN,skincareroutine,155600000000,NaN
12268,7456398001754443030,NaN,5-step rice skincare 🍚🤍 @anua_global Rice Enzy...,en,2025-01-05T11:52:32.000Z,False,blueyoobin,🦋\nkoreanskincare with me\n🇰🇷📍Paris\nyoobinblu...,58900,22,...,15700,7500000,125800,3247,False,NaN,NaN,skincareroutine,155600000000,NaN
12269,7456398001754443030,NaN,5-step rice skincare 🍚🤍 @anua_global Rice Enzy...,en,2025-01-05T11:52:32.000Z,False,blueyoobin,🦋\nkoreanskincare with me\n🇰🇷📍Paris\nyoobinblu...,58900,22,...,15700,7500000,125800,3247,False,NaN,NaN,skincareroutine,155600000000,NaN
